In [ ]:
import os
import numpy as np 
import pandas as pd 
import math

from matplotlib import pyplot as plt

# Read File

In [ ]:
with open("../input/hashcode-drone-delivery/busy_day.in","r") as text:
    data=text.read()
    data=data.split("\n")
    index=0
    #Simulation Parameters
    simulation_parameters=data[index].split(" ")
    index=index+1
    products_weight={"item_id":[],"weight":[]}
    
    drone_parameters={"drone_id":[],"momentary_row":[],"momentary_column":[],"max_load":[],"total_road":[]}
    
    for i in range(0,int(simulation_parameters[2])):
        drone_parameters["drone_id"].append(i)
        drone_parameters["momentary_row"].append(0)
        drone_parameters["momentary_column"].append(0)
        drone_parameters["max_load"].append(int(simulation_parameters[4]))
        drone_parameters["total_road"].append(0)

    #Product Parameters
    for i in range(0,int(data[index])):
        products_weight["item_id"].append(i)


    
    
    index=index+1
    for i in data[index].split(" "):
        products_weight["weight"].append(int(i))
        


    #Warehouses Parameters
    index=index+1

    warehouses_number=int(data[index])
    warehouses={"warehouses_id":[],"location_row":[],"location_column":[]}
    for i in products_weight["item_id"]:
        warehouses[i]=[]

    index=index+1
    fix_index=index
    for i in range(0,warehouses_number):
        index=fix_index+i*2
        warehouses["warehouses_id"].append(i)
        warehouses["location_row"].append(data[index].split(" ")[0])
        warehouses["location_column"].append(data[index].split(" ")[1])
        items_number=data[index+1].split(" ")
        k=0
        for j in items_number:
            warehouses[products_weight["item_id"][k]].append(int(j))
            k=k+1

    #Customer Orders Parameters
    index=index+2

    customer_orders_number=int(data[index])

    index=index+1

    customer={"customer_id":[],"location_row":[],"location_column":[],"customer_number":[]}
    customer_orders={"order_id":[],"customer_id":[],"order":[]}

    fix_index=index
    k=0
    for i in range(0,customer_orders_number):
        index=fix_index+i*3
        customer["customer_id"].append(i)
        customer["location_row"].append(data[index].split(" ")[0])
        customer["location_column"].append(data[index].split(" ")[1])
        customer["customer_number"].append(data[index+1])

        for j in data[index+2].split(" "):
            k=k+1
            customer_orders["order_id"].append(k)
            customer_orders["customer_id"].append(i)
            customer_orders["order"].append(j)


# Create DataFrame

In [ ]:
df_products_weight=pd.DataFrame(products_weight)
df_warehouses=pd.DataFrame(warehouses)
df_customer=pd.DataFrame(customer)
df_customer_orders=pd.DataFrame(customer_orders)
df_drone=pd.DataFrame(drone_parameters)


In [ ]:
#Products Weight
df_products_weight

In [ ]:
#Warehouses
df_warehouses

In [ ]:
#custormers
df_customer

In [ ]:
#Customer Orders
df_customer_orders

In [ ]:
#drone dataframe
df_drone

# Function

In [ ]:
def Find_order(order):
    a=df_warehouses[df_warehouses[order]>0][["warehouses_id","location_row","location_column",order]]
    a[order]=pd.to_numeric(a[order])
    return a

def Find_customer_orders(customer):
    df_orders=pd.DataFrame()
    for i in list(df_customer_orders[df_customer_orders["customer_id"]==customer]["order"]):
        df_orders=df_orders.append(Find_order(int(i)))
    return df_orders



def Find_min_length(drone_id,df_orders):
    drone_row=int(df_drone[df_drone["drone_id"]==drone_id]["momentary_row"])
    drone_column=int(df_drone[df_drone["drone_id"]==drone_id]["momentary_column"])
    length={}
    for i in list(df_orders["warehouses_id"].unique()):
        row=int(df_warehouses[df_warehouses["warehouses_id"]==i]["location_row"])
        column=int(df_warehouses[df_warehouses["warehouses_id"]==i]["location_column"])
        length[i]=(((drone_row-row)**2+(drone_column-column)**2)**0.5)
    
    for i,j in length.items():
        if j==min(length.values()):
            w=[i,math.ceil(j)]
    
    return w

def Find_length(customer_id,warehouses_id):
    customer_row=int(df_customer[df_customer["customer_id"]==customer_id]["location_row"])
    customer_column=int(df_customer[df_customer["customer_id"]==customer_id]["location_column"])
    
    warehouses_row=int(df_warehouses[df_warehouses["warehouses_id"]==warehouses_id]["location_row"])
    warehouses_column=int(df_warehouses[df_warehouses["warehouses_id"]==warehouses_id]["location_column"])
    length=(((customer_row-warehouses_row)**2+(customer_column-warehouses_column)**2)**0.5)
    
    
    
    return length

def Fly_drone(drone_id,customer_id):
    index=df_drone[df_drone["drone_id"]==drone_id].index
    
    df_aa=df_customer_orders[df_customer_orders["customer_id"]==customer_id]
    total_weight=int(df_drone.loc[index,"max_load"])
    
    last_condition=True
    go_customer=False
    
    df_basket=pd.DataFrame(columns=["warehouses_id","item_id","item_weight","total_item"])
    commands=[]
    
    k=0
    while last_condition:
        
        df_warehouse=pd.DataFrame(columns=["item_id","item_total","item_weight"])
        df_orders=Find_customer_orders(customer_id)
        w=Find_min_length(drone_id,df_orders)
        
        
        df_bb=df_orders[df_orders["warehouses_id"]==w[0]]
        df_bb=df_bb.groupby("warehouses_id").sum()
        j=0
        new_row=int(df_warehouses[df_warehouses["warehouses_id"]==w[0]]["location_row"])
        new_column=int(df_warehouses[df_warehouses["warehouses_id"]==w[0]]["location_column"])
        
        
        for i in list(df_bb.columns):
            if int(df_bb[i])!=0:
                df_warehouse.loc[j]=[i,int(df_bb[i]),int(df_products_weight[df_products_weight["item_id"]==i]["weight"])]
                j=j+1
        df_warehouse=df_warehouse.sort_values(["item_weight","item_id"])
        
        
        if (total_weight-df_warehouse["item_weight"].min())>=0:
            df_drone.loc[index,"momentary_row"]=new_row
            df_drone.loc[index,"momentary_column"]=new_column
            df_drone.loc[index,"total_road"]=df_drone.loc[index,"total_road"]+w[1]
            
            for i,r in df_warehouse.iterrows():
                if (total_weight-r["item_weight"])>=0:
                    if r["item_id"] in list(df_basket["item_id"]):
                        bi1=df_basket[df_basket["item_id"]==r["item_id"]].index
                        df_basket.loc[bi1,"total_item"]=int(df_basket.loc[bi1,"total_item"])+1
                        
                    else:
                        df_basket.loc[k]=[w[0],r["item_id"],r["item_weight"],1]
                   
                    bi2=df_basket[df_basket["item_id"]==r["item_id"]].index
                    
                    
                    
                    total_weight=total_weight-r["item_weight"]
    
                    
                    ai=df_aa[df_aa["order"]==str(r["item_id"])].index
                    
                    o_id=df_customer_orders[(df_customer_orders["order"]==str(r["item_id"])) & (df_customer_orders["customer_id"]==customer_id)]["order_id"].head(1)
                    o=o_id.index
        
                    
                    df_customer_orders.drop(o,inplace=True)
                    
                    
                    wi=df_warehouses[df_warehouses["warehouses_id"]==w[0]].index
                    df_warehouses.loc[wi,r["item_id"]]=int(df_warehouses.loc[wi,r["item_id"]])-1
                    
                    
                    df_aa=df_aa.drop(ai[0])
           
                    
                k=k+1
                                   
                    
                
        else:
            
                           
            
            
            go_customer=True
        
        
            
            
        if len(df_aa)==0:
            go_customer=True
            last_condition=False
            
        if go_customer:
            new_row=int(df_customer[df_customer["customer_id"]==customer_id]["location_row"])
            new_column=int(df_customer[df_customer["customer_id"]==customer_id]["location_column"])
            
            row=df_drone.loc[index,"momentary_row"]
            column=df_drone.loc[index,"momentary_column"]
            
            road=(((new_row-row)**2+(new_column-column)**2)**0.5)
            
            
            
            df_drone.loc[index,"momentary_row"]=new_row
            df_drone.loc[index,"momentary_column"]=new_column
            df_drone.loc[index,"total_road"]=df_drone.loc[index,"total_road"]+road
            
            
            total_weight=int(df_drone.loc[index,"max_load"])
            
            for i in df_basket.index:
                command=str(drone_id)+" "+"L"+" "+str(df_basket.loc[i,"warehouses_id"])+" "+str(df_basket.loc[i,"item_id"])+" "+str(df_basket.loc[i,"total_item"])
                commands.append(command)
            
            for i in df_basket.index:
                
                command=str(drone_id)+" "+"D"+" "+str(customer_id)+" "+str(df_basket.loc[i,"item_id"])+" "+str(df_basket.loc[i,"total_item"])
                commands.append(command)
                
                df_basket.drop(i,inplace=True)
            
            go_customer=False
                
            
        
        
   
        
    return commands 
    

In [ ]:
df_test=Find_customer_orders(0)
w=Find_min_length(0,df_test)
w



In [ ]:
def Find_min_length2(drone_id,df_basket):
    w=[]
    drone_row=int(df_drone[df_drone["drone_id"]==drone_id]["momentary_row"])
    drone_column=int(df_drone[df_drone["drone_id"]==drone_id]["momentary_column"])
    length={}
    for i in list(df_basket["customer_id"].unique()):
        row=int(df_customer[df_customer["customer_id"]==i]["location_row"])
        column=int(df_customer[df_customer["customer_id"]==i]["location_column"])
        length[i]=(((drone_row-row)**2+(drone_column-column)**2)**0.5)
    
    for i,j in length.items():
        if j==min(length.values()):
            w=[i,math.ceil(j)]
    
    return w

def Fly_drone2(drone_id):
    
    df_aa=pd.DataFrame()
    
    
    
    df_basket=pd.DataFrame(columns=["warehouses_id","item_id","item_weight","customer_id","total_item"])
    
    commands=[]
    
    index=df_drone[df_drone["drone_id"]==drone_id].index
    total_weight=int(df_drone.loc[index,"max_load"])
    
    go_customer=False
    last_condition=True
    
    for i in list(df_customer[df_customer["customer_number"]=="1"]["customer_id"]):
        df_aa=df_aa.append(df_customer_orders[df_customer_orders["customer_id"]==i])
    
    
    
    k=0
    while last_condition:
        df_warehouse=pd.DataFrame(columns=["item_id","item_total","item_weight"])
        df_orders=pd.DataFrame()
        
        
        for i in list(df_aa["customer_id"]):
            df_orders=df_orders.append(Find_customer_orders(i))
        w=Find_min_length(drone_id,df_orders)
        
        
        
        df_bb=df_orders[df_orders["warehouses_id"]==w[0]]
        df_bb=df_bb.groupby("warehouses_id").sum()
        
        new_row=int(df_warehouses[df_warehouses["warehouses_id"]==w[0]]["location_row"])
        new_column=int(df_warehouses[df_warehouses["warehouses_id"]==w[0]]["location_column"])
        
        j=0
        
        for i in list(df_bb.columns):
            if int(df_bb[i])!=0:
                df_warehouse.loc[j]=[i,int(df_bb[i]),int(df_products_weight[df_products_weight["item_id"]==i]["weight"])]
                j=j+1
        df_warehouse=df_warehouse.sort_values(["item_weight","item_id"])
        
        
        if (total_weight-df_warehouse["item_weight"].min())>=0:
            df_drone.loc[index,"momentary_row"]=new_row
            df_drone.loc[index,"momentary_column"]=new_column
            df_drone.loc[index,"total_road"]=df_drone.loc[index,"total_road"]+w[1]
            
            for i,r in df_warehouse.iterrows():
                if (total_weight-r["item_weight"])>=0:
                    c_id=df_aa[df_aa["order"]==str(r["item_id"])]["customer_id"]
                    
                    
                    for c in c_id:
                        total_weight=total_weight-r["item_weight"]
                        if total_weight<0:
                            break
                        
                        
                        df_basket.loc[k]=[w[0],r["item_id"],r["item_weight"],int(c),1]
                        
                        
                        
                    
                        ai=df_aa[df_aa["customer_id"]==c].index
                    
                    
                    
                        customer_id=df_aa.loc[ai[0],"customer_id"]
                    
                    

                        o_id=df_customer_orders[(df_customer_orders["order"]==str(r["item_id"])) & (df_customer_orders["customer_id"]==customer_id)]["order_id"].head(1)
                        o=o_id.index

                        df_customer_orders.drop(o,inplace=True)

                    
                        wi=df_warehouses[df_warehouses["warehouses_id"]==w[0]].index
                        df_warehouses.loc[wi,r["item_id"]]=int(df_warehouses.loc[wi,r["item_id"]])-1

                        df_aa=df_aa.drop(ai[0])
                        
                        k=k+1
                        
        else:
            go_customer=True
        
        if len(df_aa)==0:
            go_customer=True
            last_condition=False
                
        if go_customer:
            total_weight=int(df_drone.loc[index,"max_load"])
            
            df_load=df_basket[["warehouses_id","item_id","item_weight","total_item"]]
            df_load=df_load.groupby(["warehouses_id","item_id","item_weight"]).sum()
            
           
            
            
            for i in df_load.index:
               
                command=str(drone_id)+" "+"L"+" "+str(i[0])+" "+str(i[1])+" "+str(df_load.loc[i,"total_item"])
                commands.append(command)
             
            delivery=True
            while delivery:
        
                w2=Find_min_length2(drone_id,df_basket)

                new_row2=int(df_customer[df_customer["customer_id"]==w2[0]]["location_row"])
                new_column2=int(df_customer[df_customer["customer_id"]==w2[0]]["location_column"])

                
                      
                df_drone.loc[index,"momentary_row"]=new_row
                df_drone.loc[index,"momentary_column"]=new_column
                df_drone.loc[index,"total_road"]=df_drone.loc[index,"total_road"]+w2[1]
                
                di=df_basket[df_basket["customer_id"]==w2[0]].index
                command=str(drone_id)+" "+"D"+" "+str(int(df_basket.loc[di,"customer_id"]))+" "+str(int(df_basket.loc[di,"item_id"]))+" "+str(int(df_basket.loc[di,"total_item"]))
                commands.append(command)
                
                df_basket.drop(di,inplace=True)
                
                if len(df_basket)==0:
                    delivery=False
                
                  
            
            
            
            
            
            go_customer=False
                
                
        
    return commands

In [ ]:
def Fly_load(drone_id,customer_id,total_weight):
    index=df_drone[df_drone["drone_id"]==drone_id].index
    
    df_aa=df_customer_orders[df_customer_orders["customer_id"]==customer_id]
    
    last_condition=True
    
    
    df_basket=pd.DataFrame(columns=["warehouses_id","item_id","item_weight","customer_id","total_item"])
    
    
    k=0
    while last_condition:
        
        df_warehouse=pd.DataFrame(columns=["item_id","item_total","item_weight"])
        df_orders=Find_customer_orders(customer_id)
        w=Find_min_length(drone_id,df_orders)
        
        
        df_bb=df_orders[df_orders["warehouses_id"]==w[0]]
        df_bb=df_bb.groupby(level=0).first()
        df_bb=df_bb.drop(columns=["warehouses_id","location_row","location_column"])
        df_bb=df_bb.fillna(0)
        j=0
        new_row=int(df_warehouses[df_warehouses["warehouses_id"]==w[0]]["location_row"])
        new_column=int(df_warehouses[df_warehouses["warehouses_id"]==w[0]]["location_column"])
        
    
        for i in list(df_bb.columns):
            if int(df_bb[i])!=0:
                df_warehouse.loc[j]=[i,int(df_bb[i]),int(df_products_weight[df_products_weight["item_id"]==i]["weight"])]
                j=j+1
        df_warehouse=df_warehouse.sort_values(["item_weight","item_id"])
        
     
        
        if (total_weight-df_warehouse["item_weight"].min())>=0:
            df_drone.loc[index,"momentary_row"]=new_row
            df_drone.loc[index,"momentary_column"]=new_column
            df_drone.loc[index,"total_road"]=df_drone.loc[index,"total_road"]+w[1]
            
            
            for i,r in df_warehouse.iterrows():
                if (total_weight-r["item_weight"])>=0 and r["item_total"]!=0:
                    bi1=df_basket[df_basket["item_id"]==r["item_id"]].index
                                        
                    if r["item_id"] in list(df_basket["item_id"]) and w[0]==int(df_basket.loc[bi1,"warehouses_id"]):
                        df_basket.loc[bi1,"total_item"]=int(df_basket.loc[bi1,"total_item"])+1
                        
                    else:
                        df_basket.loc[k]=[w[0],r["item_id"],r["item_weight"],customer_id,1]
                   
                    bi2=df_basket[df_basket["item_id"]==r["item_id"]].index
                    
                    
                    
                    total_weight=total_weight-r["item_weight"]
    
                    
                    ai=df_aa[df_aa["order"]==str(r["item_id"])].index
                    
                    o_id=df_customer_orders[(df_customer_orders["order"]==str(r["item_id"])) & (df_customer_orders["customer_id"]==customer_id)]["order_id"].head(1)
                    o=o_id.index
        
                    
                    df_customer_orders.drop(o,inplace=True)
                    
                    
                    wi=df_warehouses[df_warehouses["warehouses_id"]==w[0]].index
                    df_warehouses.loc[wi,r["item_id"]]=int(df_warehouses.loc[wi,r["item_id"]])-1
                    
                    
                    df_aa=df_aa.drop(ai[0])
           
                    
                k=k+1
                                   
                       
        else:                 
            
            last_condition=False
        
        if len(df_aa)==0:
            last_condition=False
            
        
   
        
    return df_basket

def Fly_deliver(drone_id,df_basket):
    commands=[]
    index=df_drone[df_drone["drone_id"]==drone_id].index
    df_load=df_basket[["warehouses_id","item_id","item_weight","total_item"]]
    df_load=df_load.groupby(["warehouses_id","item_id","item_weight"],sort=False).sum()
            
           
            
     
    for i in df_load.index:
               
        command=str(drone_id)+" "+"L"+" "+str(i[0])+" "+str(i[1])+" "+str(df_load.loc[i,"total_item"])
        commands.append(command)
            
   
    delivery=True
    while delivery:
        
        w2=Find_min_length2(drone_id,df_basket)

        new_row=int(df_customer[df_customer["customer_id"]==w2[0]]["location_row"])
        new_column=int(df_customer[df_customer["customer_id"]==w2[0]]["location_column"])

                
                      
        df_drone.loc[index,"momentary_row"]=new_row
        df_drone.loc[index,"momentary_column"]=new_column
        df_drone.loc[index,"total_road"]=df_drone.loc[index,"total_road"]+w2[1]
                
        d_index=df_basket[df_basket["customer_id"]==w2[0]].index
        di=d_index[0]
        
        command=str(drone_id)+" "+"D"+" "+str(int(df_basket.loc[di,"customer_id"]))+" "+str(int(df_basket.loc[di,"item_id"]))+" "+str(int(df_basket.loc[di,"total_item"]))
        commands.append(command)
                
        df_basket.drop(di,inplace=True)
                
        if len(df_basket)==0:
            delivery=False
    
    return commands

def Find_min_weight(customer_id):
    weight={}
    m=[]
    for i in list(df_customer_orders[df_customer_orders["customer_id"]==customer_id]["order"]):
        wi=df_products_weight[df_products_weight["item_id"]==int(i)].index
        weight[i]=int(df_products_weight.loc[wi,"weight"])
    for i,j in weight.items():
        if j==min(weight.values()):
            m=[i,j]
        
    return m

In [ ]:
df_warehouses

In [ ]:
df_customer

In [ ]:
df_customer2=df_customer.copy()

In [ ]:
df_customer2["customer_id"]

In [ ]:
df_customer2=df_customer.copy()
for i in list(df_warehouses["warehouses_id"]):
    df_customer2[i]=df_customer2["customer_id"].apply(Find_length,args=(i,))

In [ ]:
df_customer2

In [ ]:
df_basket=pd.DataFrame()
a=[]
i=0


while len(df_customer_orders)!=0:
    i=int(df_customer_orders["customer_id"].head(1))
    d_id=list(df_drone.sort_values(["total_road","drone_id"])["drone_id"])[0]
    di=df_drone[df_drone["drone_id"]==d_id].index
    total_weight=int(df_drone.loc[di,"max_load"])
    
    df_basket=Fly_load(d_id,i,total_weight)
    s=0
    for k in df_basket["item_weight"]*df_basket["total_item"]:
            s=s+k
    total_weight=total_weight-s

    w_id=int(df_basket["warehouses_id"].tail(1))
    last_warehouse=df_warehouses[df_warehouses["warehouses_id"]==w_id]
    
    print(i)
    

    item_list=[]
    suitable_items=list(df_products_weight[df_products_weight["weight"]<total_weight]["item_id"])

    customers=[]

    for item in suitable_items:
        if int(last_warehouse[item])!=0:
            customers.extend(list(df_customer_orders[df_customer_orders["order"]==str(item)]["customer_id"]))
    
    customers=set(customers)
    customers=customers.union(customers)
    
    ni=list(df_customer2[df_customer2["customer_id"].isin(customers)].sort_values(w_id)["customer_id"])
    
    
    if len(customers)!=0:
        for j in ni:
            
                        
            try:                
                add_basket=Fly_load(d_id,j,total_weight)
            except:
                break
            if len(add_basket)!=0:
                df_basket=df_basket.append(add_basket)
                df_basket.index=range(0,len(df_basket))
                s=0
                for k in df_basket["item_weight"]*df_basket["total_item"]:
                    s=s+k
                total_weight=int(df_drone.loc[di,"max_load"])
                total_weight=total_weight-s
                
            else:
                break
            k=k+1
        
    
    
    a.extend(Fly_deliver(d_id,df_basket))
    

In [ ]:
a

In [ ]:
customers

In [ ]:
ni

# **Final 6.0**

In [ ]:
"""
df_basket=pd.DataFrame()
a=[]


while len(df_customer_orders)!=0:
    i=int(df_customer_orders["customer_id"].head(1))
    d_id=list(df_drone.sort_values(["total_road","drone_id"])["drone_id"])[0]
    di=df_drone[df_drone["drone_id"]==d_id].index
    total_weight=int(df_drone.loc[di,"max_load"])
    
    df_basket=Fly_load(d_id,i,total_weight)
    s=0
    for k in df_basket["item_weight"]*df_basket["total_item"]:
            s=s+k
    

    w_id=int(df_basket["warehouses_id"].tail(1))
    last_warehouse=df_warehouses[df_warehouses["warehouses_id"]==w_id]
    total_weight=total_weight-s
    print(i)
    

    item_list=[]
    suitable_items=list(df_products_weight[df_products_weight["weight"]<total_weight]["item_id"])

    customers=[]

    for item in suitable_items:
        if int(last_warehouse[item])!=0:
            customers.extend(list(df_customer_orders[df_customer_orders["order"]==str(item)]["customer_id"]))
    customers=pd.DataFrame(customers,columns=["customer_id"])
    ni=customers.value_counts("customer_id",sort=True).head(1)
    if len(ni)!=0:
        ni=ni.index[0]

        df_basket=df_basket.append(Fly_load(d_id,ni,total_weight))
        df_basket.index=range(0,len(df_basket))
    
    
    a.extend(Fly_deliver(d_id,df_basket))
"""

# **Final 5.5**

In [ ]:
"""
df_basket=pd.DataFrame()
i=0
a=[]
while len(df_customer_orders)!=0:
    d_id=list(df_drone.sort_values(["total_road","drone_id"])["drone_id"])[0]
    di=df_drone[df_drone["drone_id"]==d_id].index
    total_weight=int(df_drone.loc[di,"max_load"])
    df_aa=df_customer_orders[df_customer_orders["customer_id"]==i]
    
    if len(df_aa)!=0:
        df_basket=Fly_load(d_id,i,total_weight)
        
        
        s=0
        for k in df_basket["item_weight"]*df_basket["total_item"]:
            s=s+k
        ni=i+1
        try:
            m=Find_min_weight(ni)
            while len(m)==0:
                ni=ni+1
                m=Find_min_weight(ni)
                if ni>len(df_customer):
                    break
            
            if (s+m[1])<int(df_drone.loc[di,"max_load"]):
                total_weight=total_weight-s
                df_basket=df_basket.append(Fly_load(d_id,ni,total_weight))
                df_basket.index=range(0,len(df_basket))
            
        except:
            pass
        a.extend(Fly_deliver(d_id,df_basket))
        
        
    else:
        i=i+1
    print(i)
    
a
"""

In [ ]:
#a=[]


#for i in list(df_customer["customer_id"]):
#   d_id=list(df_drone.sort_values(["total_road","drone_id"])["drone_id"])[0]
#   a.extend(Fly_drone(d_id,i))
    


In [ ]:
#a=[]
#d_id=-1

#for i in list(df_customer["customer_id"]):
#    if i%42==0:
#        d_id=d_id+1
#    a.extend(Fly_drone(d_id,i))
              

In [ ]:
#a=[]
#for i in list(df_customer["customer_id"]):
#    drone=[]
#    for d in df_drone.index:
#        w=Find_min_length(d,Find_customer_orders(i))
#        drone.append(df_drone.loc[d,"total_road"]+w[1])
#    d_id=drone.index(min(drone))
#    a.extend(Fly_drone(d_id,i))

In [ ]:
#a=[]

#a.extend(Fly_drone2(0))

#for i in list(df_customer[df_customer["customer_number"]!="1"]["customer_id"]):
#  d_id=list(df_drone.sort_values(["total_road","drone_id"])["drone_id"])[0]
#   a.extend(Fly_drone(d_id,i))

# submission

In [ ]:
df_submission=pd.DataFrame(a,columns=[len(a)])
df_submission.to_csv("./submission.csv",index=False)

In [ ]:
df_submission

In [ ]:
df_drone